LOAD YOUR DATA

In [ ]:
import os
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
import cv2
from PIL import Image
path = "/path/to/images"
images = []
labels = []
for filename in os.listdir(path):
    img_path = os.path.join(path, filename)
    img = Image.open(img_path)
    if 'overlap' in filename:
      labels.append(1)
    elif 'non_overlap' in filename:
      labels.append(2)
    elif 'useless' in filename:
      labels.append(3)
    size = (224 ,224)
    image = img.resize(size)
    # plt.imshow(image, cmap="gray")
    # plt.axis('off')
    # plt.show()
    image = np.array(image)
    image = preprocess_input(image)
    images.append(image)

In [ ]:
images = np.array(images)
images = np.stack(images, axis=0)
print(len(labels))
print(labels)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit the LabelEncoder to your class labels
label_encoder.fit(labels)

# Get the encoded labels
labels = label_encoder.transform(labels)

# Print the encoded labels
labels.shape

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=23, stratify=labels)

print("Train data shape:", train_images.shape)
print("Train labels shape:", train_labels.shape)
print("Validation data shape:", val_images.shape)
print("Validation labels shape:", val_labels.shape)
# print("Test data shape:", test_images.shape)
# print("Test labels shape:", test_labels.shape)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l1_l2, l2, l1
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, AveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

model = keras.Sequential([
        base_model,
        layers.BatchNormalization(),
        layers.AveragePooling2D(pool_size=(2, 2), padding='same'),
        layers.Flatten(),
        layers.Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
        layers.Dropout(0.2),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(3, activation='softmax')])

for layer in base_model.layers:
  layer.trainable = False
for layer in base_model.layers[-10:]:
    layer.trainable = True
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_images, train_labels, batch_size=64, epochs=100, validation_data=(val_images, val_labels), callbacks=[early_stopping])
model.save('/path/to/model.h5')

In [ ]:
import matplotlib.pyplot as plt

train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

epochs = range(1, len(train_loss) + 1)
# plt.plot(epochs, train_loss, 'b-', label='Training Loss')
# plt.plot(epochs, val_loss, 'r-', label='Validation Loss')
plt.plot(epochs, train_accuracy, 'b--', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r--', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Metrics')
plt.title('Training and Validation Metrics')
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.optimizers import Adam
early_stopping = EarlyStopping(monitor='val_accuracy', patience=6, restore_best_weights=True)
for layer in base_model.layers:
    layer.trainable = True

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels, batch_size=64, epochs=100, validation_data=(val_images, val_labels), callbacks=[early_stopping])
model.save('/path/to/fine_tuned_model.h5')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

test_predictions = model.predict(val_images)
predicted_labels = np.argmax(test_predictions, axis=1)
true_labels = val_labels

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print("Accuracy:", accuracy)
# Calculate precision
precision = precision_score(true_labels, predicted_labels, average='weighted')
print("Precision:", precision)

# Calculate recall
recall = recall_score(true_labels, predicted_labels, average='weighted')
print("Recall:", recall)

# Calculate F1-score
f1 = f1_score(true_labels, predicted_labels, average='weighted')
print("F1-score:", f1)

In [ ]:
import matplotlib.pyplot as plt

train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'b-', label='Training Loss')
plt.plot(epochs, val_loss, 'r-', label='Validation Loss')
plt.plot(epochs, train_accuracy, 'b--', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r--', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Metrics')
plt.title('Training and Validation Metrics')
plt.legend()
plt.show()